In [2]:
import pandas as pd
import numpy as np

In [4]:
billboard_data1= pd.DataFrame(columns=['Title', 'Artist', 'Year'])

In [5]:
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,2014, 2015, 2016, 2017, 2018]
url_template = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_'

In [15]:
url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2000'
temp_df = pd.read_html(url, header=0)[0]

In [16]:
temp_df.head()

,No.,Title,Artist(s)
0,1,"""Breathe""",Faith Hill
1,2,"""Smooth""",Santana featuring Rob Thomas
2,3,"""Maria Maria""",Santana featuring The Product G&B
3,4,"""I Wanna Know""",Joe
4,5,"""Everything You Want""",Vertical Horizon


In [18]:
for year in years:
    url = url_template + str(year)
    temp_df = pd.read_html(url, header=0)[0]
    temp_df.columns.values[2] = "Artist"
    temp_df = temp_df.drop(temp_df.columns[0], axis=1)
    temp_df['Artist']=temp_df['Artist'].astype(str)
    temp_df.Artist = [artist.split(' featuring')[0] for artist in temp_df.Artist]
    temp_df.Title = [title.strip('\"') for title in temp_df.Title]
    temp_df['Year'] = year
    billboard_data1 = billboard_data1.append(temp_df)

In [24]:
billboard_data1.head()

,Title,Artist,Year
0,Breathe,Faith Hill,2000
1,Smooth,Santana,2000
2,Maria Maria,Santana,2000
3,I Wanna Know,Joe,2000
4,Everything You Want,Vertical Horizon,2000


In [23]:
billboard_data1 = billboard_data1.reset_index(drop=True)
billboard_data1['Year'] = pd.to_numeric(billboard_data1['Year'])
billboard_data1['Title'] = billboard_data1['Title'].astype(str)
billboard_data1['Artist'] = billboard_data1['Artist'].astype(str)

In [26]:
billboard_data1.to_csv('C:\\Users\\DELL\\Desktop\\minor-project\\billboard_data1.csv')

In [64]:
billboard_data1=pd.read_csv('C:\\Users\\DELL\\Desktop\\minor-project\\data-files\\billboard_data1.csv',index_col=0)
billboard_data1=billboard_data1.drop('Year',axis=1)
billboard_data1 = billboard_data1.drop_duplicates(subset="Title", keep="first")

In [65]:
#Data integration
spotify_songs = pd.read_csv('C:\\Users\\DELL\\Desktop\\minor-project\\data-files\\song_info.csv')

In [66]:
spotify_songs.head()

,song_name,artist_name,album_names,playlist
0,Boulevard of Broken Dreams,Green Day,Greatest Hits: God's Favorite Band,00s Rock Anthems
1,In The End,Linkin Park,Hybrid Theory,00s Rock Anthems
2,Seven Nation Army,The White Stripes,Elephant,00s Rock Anthems
3,By The Way,Red Hot Chili Peppers,By The Way (Deluxe Version),00s Rock Anthems
4,How You Remind Me,Nickelback,Silver Side Up,00s Rock Anthems


In [67]:
spotify_songs = spotify_songs.drop(['album_names','playlist'], axis=1)

In [68]:
spotify_songs.head()

,song_name,artist_name
0,Boulevard of Broken Dreams,Green Day
1,In The End,Linkin Park
2,Seven Nation Army,The White Stripes
3,By The Way,Red Hot Chili Peppers
4,How You Remind Me,Nickelback


In [69]:
spotify_songs.columns.values

array(['song_name', 'artist_name'], dtype=object)

In [70]:
spotify_songs = spotify_songs.rename(columns={"song_name": "Title", "artist_name": "Artist"})

In [71]:
spotify_songs.head()

,Title,Artist
0,Boulevard of Broken Dreams,Green Day
1,In The End,Linkin Park
2,Seven Nation Army,The White Stripes
3,By The Way,Red Hot Chili Peppers
4,How You Remind Me,Nickelback


In [72]:
spotify_songs = spotify_songs.drop_duplicates(subset="Title", keep="first")

In [73]:
spotify_songs['Top100'] = 0
billboard_data1['Top100'] = 1

In [74]:
combined_df = pd.concat([spotify_songs, billboard_data1], ignore_index=True, sort=True)

In [75]:
combined_df = combined_df.drop_duplicates(subset="Title", keep="last")

In [76]:
combined_df.head()

,Artist,Title,Top100
1,Linkin Park,In The End,0
2,The White Stripes,Seven Nation Army,0
3,Red Hot Chili Peppers,By The Way,0
5,Evanescence,Bring Me To Life,0
6,Papa Roach,Last Resort,0


In [77]:
combined_df.tail()

,Artist,Title,Top100
14565,Daddy Yankee,Dura,1
14566,XXXTentacion,Changes,1
14567,Luke Combs,One Number Away,1
14568,Rae Sremmurd,Powerglide,1
14569,Dua Lipa,IDGAF,1


In [78]:
billboard_data1.head()

,Title,Artist,Top100
0,Breathe,Faith Hill,1
1,Smooth,Santana,1
2,Maria Maria,Santana,1
3,I Wanna Know,Joe,1
4,Everything You Want,Vertical Horizon,1


In [80]:
combined_df.to_csv('C:\\Users\\DELL\\Desktop\\minor-project\\data-files\\songs-list.csv')

In [81]:
#Getting songs uris
combined_df.shape

(13993, 3)

In [1]:
import spotipy
import spotipy.util as util
from tqdm import tqdm_notebook

In [105]:
def get_spotify_uri(title, artist):
    query = title + " " + artist
    search = sp.search(q=query, limit=50, offset=0, type='track', market='US')
    search_items = search['tracks']['items']
    for i in range(len(search_items)):
            uri = search_items[i]['id']
            return uri
    return 0

In [106]:
token = util.oauth2.SpotifyClientCredentials(client_id='b7fc404a069d4a0f83049834a7d09a25', client_secret='990a5a3792e9469fac00c288ec610d7d')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [112]:
titles = list(combined_df.Title)
artists = list(combined_df.Artist)
top100 = list(combined_df.Top100)
spotify_uri = list()
errors = list()

In [114]:
temp1 = list()

In [115]:
for i in tqdm_notebook(range(0,13993)):
    uri = get_spotify_uri(titles[i], artists[i])
    
    if uri != 0:
        temp1.append(uri)
    else:
        temp1.append(uri)
        errors.append(i)
spotify_uri = spotify_uri + temp1


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [123]:
len(spotify_uri)

13993

In [124]:
songs_with_id = pd.DataFrame()

In [126]:
songs_with_id['Title'] = titles;
songs_with_id['Artist'] = artists;
songs_with_id['Top100'] = top100;
songs_with_id['URI'] = spotify_uri;
songs_with_id=songs_with_id[songs_with_id.URI!=0]

In [127]:
songs_with_id.head()

,Title,Artist,Top100,URI
0,In The End,Linkin Park,0,7KSjlzDJvwAG0utAYE9Vvg
1,Seven Nation Army,The White Stripes,0,7i6r9KotUPQg3ozKKgEPIN
2,By The Way,Red Hot Chili Peppers,0,3ZOEytgrvLwQaqXreDs2Jx
3,Bring Me To Life,Evanescence,0,0COqiPhxzoWICwFCS4eZcp
4,Last Resort,Papa Roach,0,5W8YXBz9MTIDyrpYaCg2Ky


In [128]:
songs_with_id.tail()

,Title,Artist,Top100,URI
13988,Dura,Daddy Yankee,1,6KuqAtoeVzxAYOaMveLNpH
13989,Changes,XXXTentacion,1,7AFASza1mXqntmGtbxXprO
13990,One Number Away,Luke Combs,1,4gB7HrYHbJVJ5RFOjxmoq4
13991,Powerglide,Rae Sremmurd,1,1BuZAIO8WZpavWVbbq3Lci
13992,IDGAF,Dua Lipa,1,1f7HfxcJ151yN485mZhhWn


In [131]:
songs_with_id.shape

(13507, 4)

In [133]:
songs_with_id.reset_index(drop=True,inplace=True)

In [134]:
songs_with_id.tail()

,Title,Artist,Top100,URI
13502,Dura,Daddy Yankee,1,6KuqAtoeVzxAYOaMveLNpH
13503,Changes,XXXTentacion,1,7AFASza1mXqntmGtbxXprO
13504,One Number Away,Luke Combs,1,4gB7HrYHbJVJ5RFOjxmoq4
13505,Powerglide,Rae Sremmurd,1,1BuZAIO8WZpavWVbbq3Lci
13506,IDGAF,Dua Lipa,1,1f7HfxcJ151yN485mZhhWn


In [135]:
songs_with_id.to_csv('C:\\Users\\DELL\\Desktop\\minor-project\\data-files\\songs-with-id.csv')

In [136]:
#Getting song features
uris = list(songs_with_id.URI)
danceability_list = list()
energy_list = list()
key_list = list()
loudness_list = list()
mode_list = list()
speechiness_list = list()
acousticness_list = list()
instrumentalness_list = list()
liveness_list = list()
valence_list = list()
tempo_list = list()
duration_list = list()
time_signature_list= list()

In [137]:
def get_audio_features(uri):
    search = sp.audio_features(uri)
    if search[0] == None:
        danceability_list.append(np.nan)
        energy_list.append(np.nan)
        key_list.append(np.nan)
        loudness_list.append(np.nan)
        mode_list.append(np.nan)
        speechiness_list.append(np.nan)
        acousticness_list.append(np.nan)
        instrumentalness_list.append(np.nan)
        liveness_list.append(np.nan)
        valence_list.append(np.nan)
        tempo_list.append(np.nan)
        duration_list.append(np.nan)
        time_signature_list.append(np.nan) 
        return ('Error on: ' + str(uri))
    
    search_list = search[0]
    
    danceability_list.append(search_list['danceability'])
    energy_list.append(search_list['energy'])
    key_list.append(search_list['key'])
    loudness_list.append(search_list['loudness'])
    mode_list.append(search_list['mode'])
    speechiness_list.append(search_list['speechiness'])
    acousticness_list.append(search_list['acousticness'])
    instrumentalness_list.append(search_list['instrumentalness'])
    liveness_list.append(search_list['liveness'])
    valence_list.append(search_list['valence'])
    tempo_list.append(search_list['tempo'])
    duration_list.append(search_list['duration_ms'])
    time_signature_list.append(search_list['time_signature'])

In [139]:
token = util.oauth2.SpotifyClientCredentials(client_id='b7fc404a069d4a0f83049834a7d09a25', client_secret='990a5a3792e9469fac00c288ec610d7d')
cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)
sp = spotipy.Spotify(auth=cache_token)

In [140]:
for i in tqdm_notebook(range(len(uris))):
    get_audio_features(uris[i])

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


In [141]:
songs_with_id['Danceability'] = danceability_list
songs_with_id['Energy'] = energy_list
songs_with_id['Key'] = key_list
songs_with_id['Loudness'] = loudness_list
songs_with_id['Mode'] = mode_list
songs_with_id['Speechiness'] = speechiness_list
songs_with_id['Acousticness'] = acousticness_list
songs_with_id['Instrumentalness'] = instrumentalness_list
songs_with_id['Liveness'] = liveness_list
songs_with_id['Valence'] = valence_list
songs_with_id['Tempo'] = tempo_list
songs_with_id['Duration'] = duration_list
songs_with_id['Time_Signature'] = time_signature_list

In [142]:
songs_with_id.dropna()

,Title,Artist,Top100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature
0,In The End,Linkin Park,0,7KSjlzDJvwAG0utAYE9Vvg,0.476,0.632,9,-6.217,0,0.0376,0.165000,0.337000,0.1110,0.1320,107.961,184787,4
1,Seven Nation Army,The White Stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0,-7.828,1,0.0792,0.008170,0.447000,0.2550,0.3240,123.881,231733,4
2,By The Way,Red Hot Chili Peppers,0,3ZOEytgrvLwQaqXreDs2Jx,0.618,0.938,9,-3.442,1,0.0456,0.017900,0.000000,0.1670,0.8750,91.455,269000,4
3,Bring Me To Life,Evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4,-3.188,0,0.0698,0.007210,0.000002,0.2420,0.2960,94.612,235893,4
4,Last Resort,Papa Roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4,-3.719,0,0.0603,0.000481,0.000820,0.2010,0.6920,90.598,199907,4
5,Sex on Fire,Kings of Leon,0,0ntQJM78wzOLVeCUAW7Y45,0.542,0.905,9,-5.653,1,0.0540,0.001720,0.010400,0.1360,0.3740,153.398,203347,4
6,Smooth Criminal,Alien Ant Farm,0,5z6xHjCZr7a7AIcy8sPBKy,0.653,0.964,9,-4.261,0,0.0582,0.003160,0.005120,0.1440,0.8700,126.928,209267,4
7,Can't Stop,Red Hot Chili Peppers,0,3ZOEytgrvLwQaqXreDs2Jx,0.618,0.938,9,-3.442,1,0.0456,0.017900,0.000000,0.1670,0.8750,91.455,269000,4
8,Chop Suey!,System Of A Down,0,2DlHlPMa4M17kufBvI2lEN,0.419,0.934,7,-3.908,0,0.1200,0.000278,0.001500,0.1320,0.2860,127.288,210240,4
9,Take Me Out,Franz Ferdinand,0,6ooluO7DiEhI1zmK94nRCM,0.278,0.676,9,-8.821,1,0.0371,0.000423,0.000899,0.1360,0.4940,104.545,237027,4


In [143]:
songs_with_id.head()

,Title,Artist,Top100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature
0,In The End,Linkin Park,0,7KSjlzDJvwAG0utAYE9Vvg,0.476,0.632,9,-6.217,0,0.0376,0.165000,0.337000,0.111,0.132,107.961,184787,4
1,Seven Nation Army,The White Stripes,0,7i6r9KotUPQg3ozKKgEPIN,0.737,0.463,0,-7.828,1,0.0792,0.008170,0.447000,0.255,0.324,123.881,231733,4
2,By The Way,Red Hot Chili Peppers,0,3ZOEytgrvLwQaqXreDs2Jx,0.618,0.938,9,-3.442,1,0.0456,0.017900,0.000000,0.167,0.875,91.455,269000,4
3,Bring Me To Life,Evanescence,0,0COqiPhxzoWICwFCS4eZcp,0.331,0.943,4,-3.188,0,0.0698,0.007210,0.000002,0.242,0.296,94.612,235893,4
4,Last Resort,Papa Roach,0,5W8YXBz9MTIDyrpYaCg2Ky,0.589,0.890,4,-3.719,0,0.0603,0.000481,0.000820,0.201,0.692,90.598,199907,4


In [144]:
songs_with_id.tail()

,Title,Artist,Top100,URI,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration,Time_Signature
13502,Dura,Daddy Yankee,1,6KuqAtoeVzxAYOaMveLNpH,0.783,0.840,1,-3.416,1,0.0510,0.17400,0.0000,0.4120,0.8390,95.001,200480,4
13503,Changes,XXXTentacion,1,7AFASza1mXqntmGtbxXprO,0.669,0.308,11,-10.068,1,0.0290,0.88300,0.0000,0.0984,0.5200,64.934,121887,4
13504,One Number Away,Luke Combs,1,4gB7HrYHbJVJ5RFOjxmoq4,0.544,0.781,6,-6.160,0,0.0311,0.00585,0.0000,0.1910,0.5270,144.075,221800,4
13505,Powerglide,Rae Sremmurd,1,1BuZAIO8WZpavWVbbq3Lci,0.713,0.831,1,-4.750,0,0.1500,0.01680,0.0000,0.1180,0.5840,173.948,332301,4
13506,IDGAF,Dua Lipa,1,1f7HfxcJ151yN485mZhhWn,0.711,0.551,4,-6.107,0,0.0433,0.31900,0.0133,0.0858,0.0637,97.002,406050,4


In [145]:
songs_with_id.to_csv('C:\\Users\\DELL\\Desktop\\minor-project\\data-files\\songs-with-features.csv')